In [1]:
import pandas as pd
import json
import StringIO
import re
import os
import requests
import subprocess

# reference compounds for accurate mass searching
reference_compounds = pd.read_pickle('/Users/Onur/repos/metatlas_reactions/workflow/database/unique_compounds_groups.pkl')

# get job data

In [2]:
base_url = 'http://localhost:8000/'
auth_url = base_url + 'admin/login'

client = requests.session()
# Retrieve the CSRF token first
client.get(auth_url,verify=False)  # sets cookie
username='admin' # Superuser goes here
password='adminpass' # Superuser password goes here
csrftoken = client.cookies['csrftoken']
login_data = {'username': username,
              'password': password,
              'csrfmiddlewaretoken': csrftoken}
client.post(auth_url, data=login_data, headers=dict(Referer=auth_url),verify=False)

r = client.get(os.path.join(base_url,'admin/ids/?filter=all&json=True'))

### Choose one job for testing

In [3]:
all_jobs = json.loads(r.text)
job_data = all_jobs[2]
job_data

{u'fields': {u'adducts_neg': u'n1',
  u'adducts_pos': u'p1,p2,p3,p4,p8',
  u'blast_cutoff': 99.0,
  u'chemnet_penalty': 7.0,
  u'email': u'oerbilgin@lbl.gov',
  u'fasta_file': u'input/2017/08/58a7e761-2bfb-42b7-89cf-8042c1ebef9e/fasta.faa',
  u'is_mass_search': True,
  u'is_tautomers': False,
  u'metabolite_file': u'input/2017/08/58a7e761-2bfb-42b7-89cf-8042c1ebef9e/metabolite.csv',
  u'network_level': 1,
  u'polarity': u'pos',
  u'ppm': 10.0,
  u'reciprocal_cutoff': 99.0,
  u'score_weight_compound': 2.0,
  u'score_weight_homology': 1.5,
  u'score_weight_reciprocal': 2.5,
  u'score_weight_rxnconnect': 0.0,
  u'uploaded_at': u'2017-08-18T14:27:46.990Z'},
 u'model': u'magi_task.magijob',
 u'pk': u'58a7e761-2bfb-42b7-89cf-8042c1ebef9e'}

# make some preprocessing decisions

## adjust filepaths
Need to coordinate with Ben and Matt if this is necessary and what the paths will be at NERSC

In [4]:
dir_root = '/project/projectdirs/metatlas/projects/magi_tasks'

job_data['fields']['fasta_file'] = os.path.join(dir_root, job_data['fields']['fasta_file'].split('input/')[1])
job_data['fields']['metabolite_file'] = os.path.join(dir_root, job_data['fields']['metabolite_file'].split('input/')[1])

## preprocessing functions

In [61]:
def protein_translate(seq, warnings=False):
    """
    Translates an input DNA or RNA sequence to amino acid

    Inputs
    ------
    seq: DNA or RNA sequence
    warnings: print out warnings or not
    na: nucleic acid language('dna' or 'rna')
    
    Outputs
    -------
    protein_seq: Translated protein sequence
    """
    codons = {
'AAA': 'K', 'AAC': 'N', 'AAG': 'K', 'AAT': 'N', 'ACA': 'T', 'ACC': 'T',
'ACG': 'T', 'ACT': 'T', 'AGA': 'R', 'AGC': 'S', 'AGG': 'R', 'AGT': 'S',
'ATA': 'I', 'ATC': 'I', 'ATG': 'M', 'ATT': 'I', 'CAA': 'Q', 'CAC': 'H',
'CAG': 'Q', 'CAT': 'H', 'CCA': 'P', 'CCC': 'P', 'CCG': 'P', 'CCT': 'P',
'CGA': 'R', 'CGC': 'R', 'CGG': 'R', 'CGT': 'R', 'CTA': 'L', 'CTC': 'L',
'CTG': 'L', 'CTT': 'L', 'GAA': 'E', 'GAC': 'D', 'GAG': 'E', 'GAT': 'D',
'GCA': 'A', 'GCC': 'A', 'GCG': 'A', 'GCT': 'A', 'GGA': 'G', 'GGC': 'G',
'GGG': 'G', 'GGT': 'G', 'GTA': 'V', 'GTC': 'V', 'GTG': 'V', 'GTT': 'V',
'TAA': '*', 'TAC': 'Y', 'TAG': '*', 'TAT': 'Y', 'TCA': 'S', 'TCC': 'S',
'TCG': 'S', 'TCT': 'S', 'TGA': '*', 'TGC': 'C', 'TGG': 'W', 'TGT': 'C',
'TTA': 'L', 'TTC': 'F', 'TTG': 'L', 'TTT': 'F'
        }
    # check input sequence
    if len(seq) % 3 != 0:
        raise RuntimeError('The nucelotide sequence is not a multiple of 3: %s' % (seq))
    
    # replace all Uracil with Thymine
    seq = seq.upper()
    seq = seq.replace('U', 'T')

    # make index tuples list
    cidx = range(0, len(seq)+3, 3)
    cidx = zip(cidx[:-1], cidx[1:])
    
    # translate
    protein_seq = ''
    for i, j in cidx:
        codon = seq[i:j]
        aa = codons[codon]
        protein_seq += aa
    
    # check translation for suspicious activity
    if warnings:
        if protein_seq[0] != 'M':
            print('WARNING: sequence does not start with Methionine')
        if protein_seq[-1] != '*':
            print('WARNING: sequence does not end with a STOP codon')
        if '*' in protein_seq[:-1]:
            counter = []
            for i, aa in enumerate(protein_seq[:-1]):
                if aa == '*':
                    counter.append(i * 3)
            print(
                'WARNING: sequence contains internal STOP codons at DNA positions: %s'
                 % (counter))
    
    # return the translation
    return protein_seq

def determine_fasta_language(job_data, translate=True):
    """
    Assuming this is a valid fasta file,
    determines if the fasta is protein or nucleic acid
    if translate = True, translates the file to protein if it is dna/rna
    
    Input is dict of json of one job.
    
    alters the fasta_file field in job_data json if translation occured.
    
    returns job_data
    """
    fil_path = job_data['fields']['fasta_file']
    # read fasta file
    with open(file_path, 'r') as f:
        file_data = f.read()
        
    # convert newlines
    for newline in ['\r\n', '\n\r']:
        if newline in file_data:
            file_data.replace(newline, '\n')
    if '\r' in file_data:
        file_data.replace('\r', '\n')
    
    # parse gene sequences into one long string and convert to DNA if RNA
    genes = file_data.split('>')[1:]
    seqs = ''.join([''.join(g.split('\n')[1:]).replace('-', '') for g in genes]).upper().replace('U', 'T')
    
    # determine what letters are in the gene sequences
    letters = pd.Series(list(set(seqs)))
    # first check for DNA
    if len(letters) <= 4 and letters.str.contains('[ACTG]').all():
        answer = 'dna'
    # then amino acids 
    elif letters.str.contains('[ACDEFGHIKLMNPQRSTVWY*]').all():
        answer = 'protein'
    else:
        no = letters[~letters.str.contains('[ACDEFGHIKLMNPQRSTVWY*]')].values
        raise RuntimeError('Could not determine if FASTA is nucleotide or protein. Offending character(s): %s' % (no))
    
    # translate if desired
    if translate and answer == 'dna':       
        # translate the genes
        new_data = ''
        gene_list = file_data.split('>')[1:]
        for gene in gene_list:
            header = gene.split('\n')[0]
            seq = gene.split('\n')[1:]
            seq = ''.join([i for i in seq if i != ''])
            protein = protein_translate(seq)
            new_data += '>' + header + '\n'
            new_data += protein + '\n\n'
        
        # save the new file
        new_filename = file_path.split('/')[-1].split('.')[0] + '_translated.faa'
        new_filepath = '/'.join(file_path.split('/')[:-1]) + '/%s' %(new_filename)
        with open(new_filepath, 'w') as f:
            f.write(new_data)
        # change the field in job_data
        job_data['fields']['fasta_file'] = new_filepath
        return job_data
    else:
        return job_data

def job_script(job_data):
    """
    uses job data json to create a magi job submission script for cori
    """
    account_id = 'm2650' # metatlas
    
    # where to write the job script to
    out_path = '/'.join(job_data['fields']['fasta_file'].split('/')[:-1])
    
    # need to convert this into string so we can join it later
    job_data['fields']['score_weights'] = [str(i) for i in job_data['fields']['score_weights']]
    
    job_lines = [
        '#!/bin/bash -l',
        '#SBATCH --account=%s' % (account_id),
        '#SBATCH --job-name=%s' % (job_data['job_id'].split('-')[0]),
        '#SBATCH --time=0:10:00',
        '#SBATCH --output=%s/log_out.txt' % (out_path),
        '#SBATCH --error=%s/log_err.txt' % (out_path),
        '#SBATCH --partition=debug',
        '#SBATCH --constraint=haswell',
        '#SBATCH --license=project',
        '#SBATCH --mail-user=%s' %(job_data['fields']['email']),
        '#SBATCH --mail-type=BEGIN,END,FAIL,TIME_LIMIT',
        '',
        'module load python/2.7-anaconda',
        '',
        'time python /project/projectdirs/metatlas/projects/metatlas_reactions/workflow/magi_workflow_20170519.py \\',
        '--fasta %s \\' % (job_data['fields']['fasta_file']),
        '--compounds %s \\' % (job_data['fields']['metabolite_file']),
        '--level %s \\' % (job_data['fields']['network_level']),
        # not sure if this line will break anything at nersc
        # if it does, put it at the end of the previous line
        '%s \\' % (['--tautomer' for i in [job_data['fields']['is_tautomers']] if i][0]),
        '--final_weights %s \\' % (' '.join(job_data['fields']['score_weights'])),
        '--blast_filter %s \\' % (job_data['fields']['blast_cutoff']),
        '--reciprocal_closeness %s \\' % (job_data['fields']['reciprocal_cutoff']),
        '--chemnet_penalty %s \\' % (job_data['fields']['chemnet_penalty']),
        '--output %s/output_files --mute' % (out_path),
    ]
    
    job = '\n'.join(job_lines)
    with open(os.path.join(out_path, 'job_script.sbatch'), 'w') as f:
        f.write(job)
    return 'job script written'

def ppm_window(mass, ppm=5, result='bounds'):
    """
    Given a mass and a ppm error, returns lower and upper bounds 
    corresponding to that ppm window. 
    Inputs
    ------
    mass:   monoisotopic mass
    ppm:    the ppm error to construct the window
    result: "bounds" returns a list of lower and upper bounds
            "error" returns the amu value of the error, if you wanted to
            add and/or subtract the error as you wish
    Outputs
    -------
    Either a list of lower and upper mass bounds, or a single value
    corresponding to the amu error
    """
    error = ppm/1e6 * mass
    lower_bound = mass - error
    upper_bound = mass + error
    if result.lower() == 'bounds':
        return [lower_bound, upper_bound]
    elif result.lower() == 'error':
        return error
    else:
        raise RuntimeError(
            '%s is not a valid result argument' %(result)
            )

def accurate_mass_match(mass, compound_df=None, ppm=5, extract='inchi_key'):
    """
    Accurate mass searching against a compound database.
    Inputs
    ------
    mass:           An accurate monoisotopic mass
    compound_df:    A dataframe of compounds. Must have a column named
                    "mono_isotopic_molecular_weight"
    ppm:            ppm error to allow the mass matcher
    extract:        What compound information to return. Must correspond
                    to a valid column in compound_df
    
    Outputs
    -------
    cpd:            List of compounds that were matched
    """

    err = ppm_window(mass, ppm=ppm, result='error')

    potential_compounds = compound_df[
                            abs(compound_df['mono_isotopic_molecular_weight'] \
                            - mass) <= err]
    cpds = potential_compounds[extract].values.tolist()
    if len(cpds) == 0:
        cpds = None
    return cpds

def accurate_mass_search_wrapper(job_data):
    """
    performs accurate mass search using unique_compounds table
    """

    # math table for all adducts
    adduct_table = {
        'p1' : -1.007276,
        'p2' : -18.033823,
        'p3' : -22.989218,
        'p4' : -38.963158,
        'p8' : - 42.033823,
    }

    search_ppm = job_data['fields']['ppm']

    # get adducts according to polarity
    if job_data['fields']['polarity'] == 'pos':
        adducts = job_data['fields']['adducts_pos'].split(',')
    elif job_data['fields']['polarity'] == 'pos':
        adducts = job_data['fields']['adducts_neg'].split(',')
    else:
        raise RuntimeError('Could not understand polarity')

    # load compound table (should be masses in original_compounds)
    compounds = pd.read_csv(job_data['fields']['metabolite_file'])

    # rename original_compounds column
    columns = compounds.columns.values
    columns[columns == 'original_compound'] = 'original_mz'
    compounds.columns = columns

    # set up data container
    data = {
        'original_compound': [],
        'searched_adduct': [],
        'original_mz' : []
    }
    # accurate mass search and store results
    for mz in compounds['original_mz'].unique():
        for adduct in adducts:
            neutral_mass = mz + adduct_table[adduct]
            found_compounds = accurate_mass_match(neutral_mass,
                                                  compound_df=reference_compounds,
                                                  ppm=search_ppm
                                                 )
            if found_compounds is not None:
                for cpd in found_compounds:
                    data['original_compound'].append(cpd)
                    data['searched_adduct'].append(adduct)
                    data['original_mz'].append(mz)

    # merge with user input and save
    df = pd.DataFrame(data)
    compounds = compounds.merge(df, on='original_mz', how='left')
    
    # save the new table
    new_path = job_data['fields']['metabolite_file'].split('.')[0] + '_mass_searched.csv'
    job_data['fields']['metabolite_file'] = new_path
    compounds.to_csv(job_data['fields']['metabolite_file'])

    return job_data

## First, determine language of FASTA and translate if necessary

In [6]:
print job_data['fields']['fasta_file']
fasta_language = determine_fasta_language(job_data['fields']['fasta_file'])
fasta_language

dna translated


## Second, determine if accurate mass searching needs to happen

In [62]:
print job_data['fields']['metabolite_file']
print job_data['fields']['is_mass_search']
accurate_mass_search_wrapper(job_data)

/Users/Onur/repos/magi_web/temp_magi_web/input/2017/08/58a7e761-2bfb-42b7-89cf-8042c1ebef9e/metabolite.csv
True


'accurate mass searching done'

## Last, create a job submission script

In [14]:
print job_script(job_data)

job script written
